In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [103]:
data = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [69]:
data.sentiment.value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [104]:
# remove noise and html
from bs4 import BeautifulSoup

def clean_text(text):
    soup = BeautifulSoup(text,'html.parser')
    clean_text = soup.get_text()
    clean_text = re.sub('[^\w\s]+',' ',clean_text)
    clean_text = re.sub('[\s]+',' ',clean_text)
    return clean_text
    

data.review = data.review.apply(clean_text)
data.review[0]

'One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked They are right as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence which set in right from the word GO Trust me this is not a show for the faint hearted or timid This show pulls no punches with regards to drugs sex or violence Its is hardcore in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary It focuses mainly on Emerald City an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda Em City is home to many Aryans Muslims gangstas Latinos Christians Italians Irish and more so scuffles death stares dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wouldn t dare Forget pretty

In [ ]:
# Stemming
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
def remove_stopwords_and_lemmatize(text):
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
#    stemmer = SnowballStemmer("english", ignore_stopwords=True)
    tokenizer = ToktokTokenizer()

    words = tokenizer.tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = ' '.join([lemmatizer.lemmatize(w) for w in words])
#    words = ' '.join([stemmer.stem(w) for w in words])
    return words

tokenizer = ToktokTokenizer()
words = tokenizer.tokenize(data.review[0]);
print("Before: ")
print(data.review[0])
print("After: ")
data.review = data.review.apply(remove_stopwords_and_lemmatize)
print(data.review[0])

Before: 
One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked They are right as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence which set in right from the word GO Trust me this is not a show for the faint hearted or timid This show pulls no punches with regards to drugs sex or violence Its is hardcore in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary It focuses mainly on Emerald City an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda Em City is home to many Aryans Muslims gangstas Latinos Christians Italians Irish and more so scuffles death stares dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wouldn t dare Forge

In [ ]:
from sklearn.preprocessing import LabelBinarizer

binzarizer = LabelBinarizer()
sentiment = binzarizer.fit_transform(data.sentiment)
print(sentiment.shape)
train_y = sentiment[:40000]
test_y = sentiment[40000:]
print(train_y)
print(train_y.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))

tf_train = vectorizer.fit_transform(train_X)
tf_test = vectorizer.transform(test_X)

print(tf_train.shape)
print(tf_test.shape)

In [75]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report


regression = LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)

model_tf = regression.fit(tf_train, train_y)
print(model_tf)
predict_tf = model_tf.predict(tf_test)

from sklearn.metrics import accuracy_score

print("TFIDF ACC: ")
print(accuracy_score(test_y, predict_tf))

class_names = ['Positive','Negative']


#Classification report for tfidf features
lr_tfidf_report=classification_report(test_y,predict_tf,target_names=class_names)
print(lr_tfidf_report)

predictions = [predict_tf]
for prediction in predictions:
    plot_confusion_matrix(confusion_matrix(test_y,prediction),class_names, title= "Confusion Matrix"  )


In [ ]:
from sklearn.naive_bayes import MultinomialNB


regression = MultinomialNB()

model_tf = regression.fit(tf_train, train_y)
print(model_tf)
predict_tf = model_tf.predict(tf_test)

from sklearn.metrics import accuracy_score
print("TFIDF ACC: ")
print(accuracy_score(test_y, predict_tf))


class_names = ['Positive','Negative']

#Classification report for tfidf features
tfidf_report=classification_report(test_y,predict_tf,target_names=class_names)
print(tfidf_report)



predictions = [ predict_tf]
for prediction in predictions:
    plot_confusion_matrix(confusion_matrix(test_y,prediction),class_names, title= "Confusion Matrix"  )


In [ ]:
print(tf_train)